In [2]:
from Data_loader import *

ValueError: could not broadcast input array from shape (262036,) into shape (24724,)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

In [12]:
class AdditiveAttention(nn.Module):
    def __init__(self, embed_size, attention_dim):
        
        super(AdditiveAttention, self).__init__()
        self.V_w = nn.Linear(embed_size, attention_dim)  #weight and bias (vw)
        self.q_w = nn.Parameter(torch.zeros(attention_dim))

        #initializations
        nn.init.xavier_uniform_(self.V_w.weight) 
        nn.init.zeros_(self.V_w.bias)
        nn.init.xavier_uniform_(self.q_w.unsqueeze(0)) 

    def forward(self, h):
        
        #projection into attention space and tanh after
        projection=self.V_w(h)#[batch_size, seq_length, attention_dim]
        tanh_output=torch.tanh(projection)  #[batch_size, seq_length, attention_dim]

        #attention scores and softmax to normalize scores and get weights
        attention_scores=torch.matmul(tanh_output, self.q_w)  #[batch_size, seq_length]
        attention_weights=F.softmax(attention_scores, dim=1)  #[batch_size, seq_length]

        #weighted sum of h
        attention_weights = attention_weights.unsqueeze(-1)  #[batch_size, seq_length, 1]
        r_vector=torch.sum(attention_weights * h, dim=1)  #[batch_size, embed_size]

        return r_vector

In [13]:
class NewsEncoder(nn.Module):
    def __init__(self, embed_size, heads, word_embedding_matrix,attention_dim):
        super(NewsEncoder, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(word_embedding_matrix, freeze=False)
        self.multi_head_attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads, batch_first=True)
        self.additive_attention = AdditiveAttention(embed_size, attention_dim)
        
    def forward(self, x):
        embedding = self.embedding(x)
        attn_output, _ = self.multi_head_attention(embedding, embedding, embedding)
        news_representation = self.additive_attention(attn_output)
        return news_representation

In [1]:
#test
### Test news encoder
word_embedding_matrix = glove_vectors
attention_dim=200
# Instantiate the model
encoder = NewsEncoder(embed_size=300, heads=5, word_embedding_matrix=word_embedding_matrix,attention_dim=200)

# Random input
x = input_data_train.loc[5, 'candidate_news']

tensor_list = [torch.tensor(sublist) for sublist in x]
x = pad_sequence(tensor_list, batch_first=True, padding_value=0)

# Forward pass
output = encoder(x)
print("output size", output.shape)

NameError: name 'glove_vectors' is not defined